# Baseline Bus System

Implementation of 5 busses, conventional generator at bus 1, wind generation at bus 5, dynamic load at bus 2 and ordinary demands at busses 1, 3 and 4.
 
Line connections run from:
 
5-1;
1-2;
2-3;
2-4
 
 
Extend with actual data

In [ ]:
    # For f23 and f24, we only allow positive flow (unidirectional from Bus2 to Bus3 and Bus2 to Bus4)
    @constraint(model, f23[s,t] >= 0)
    @constraint(model, f23[s,t] <= Fmax)
    @constraint(model, f24[s,t] >= 0)
    @constraint(model, f24[s,t] <= Fmax)

In [78]:
using JuMP, HiGHS

# ------------------------------
# Data and Parameters
# ------------------------------

# Time periods
T = 1:4

# Scenarios: 9 scenarios (combining 3 wind scenarios and 3 load scenarios)
scenarios = 1:9

# Scenario probabilities (equal probability for each scenario)
prob = Dict(s => 1/9 for s in scenarios)

# Wind availability at Bus 5 (stochastic, per scenario and time period)
W = Dict(
    (1,1) => 320.1, (1,2) => 299.6, (1,3) => 333.5, (1,4) => 343.3,
    (2,1) => 320.1, (2,2) => 299.6, (2,3) => 333.5, (2,4) => 343.3,
    (3,1) => 320.1, (3,2) => 299.6, (3,3) => 333.5, (3,4) => 343.3,
    (4,1) => 144.1, (4,2) => 102.2, (4,3) => 120.8, (4,4) => 162.1,
    (5,1) => 144.1, (5,2) => 102.2, (5,3) => 120.8, (5,4) => 162.1,
    (6,1) => 144.1, (6,2) => 102.2, (6,3) => 120.8, (6,4) => 162.1,
    (7,1) => 539.6, (7,2) => 629.1, (7,3) => 659.3, (7,4) => 579.8,
    (8,1) => 539.6, (8,2) => 629.1, (8,3) => 659.3, (8,4) => 579.8,
    (9,1) => 539.6, (9,2) => 629.1, (9,3) => 659.3, (9,4) => 579.8
)

# Dynamic load at Bus 1 (stochastic, per scenario and time period)
D1 = Dict(
    (1,1) => 100,  (1,2) => 90, (1,3) => 150, (1,4) => 100,
    (2,1) => 120,  (2,2) => 100, (2,3) => 180, (2,4) => 120,
    (3,1) => 200,  (3,2) => 220, (3,3) => 300, (3,4) => 200,
    (4,1) => 100,  (4,2) => 90, (4,3) => 150, (4,4) => 100,
    (5,1) => 120,  (5,2) => 100, (5,3) => 180, (5,4) => 120,
    (6,1) => 200,  (6,2) => 220, (6,3) => 300, (6,4) => 200,
    (7,1) => 100,  (7,2) => 90, (7,3) => 150, (7,4) => 100,
    (8,1) => 120,  (8,2) => 100, (8,3) => 180, (8,4) => 120,
    (9,1) => 200,  (9,2) => 220, (9,3) => 300, (9,4) => 200
)

# Dynamic load at Bus 2 (stochastic, per scenario and time period)
D2 = Dict(
    (1,1) => 50,  (1,2) => 150, (1,3) => 250, (1,4) => 75,
    (2,1) => 60,  (2,2) => 170, (2,3) => 300, (2,4) => 90,
    (3,1) => 70,  (3,2) => 190, (3,3) => 350, (3,4) => 100,
    (4,1) => 50,  (4,2) => 150, (4,3) => 250, (4,4) => 75,
    (5,1) => 60,  (5,2) => 170, (5,3) => 300, (5,4) => 90,
    (6,1) => 70,  (6,2) => 190, (6,3) => 350, (6,4) => 100,
    (7,1) => 50,  (7,2) => 150, (7,3) => 250, (7,4) => 75,
    (8,1) => 60,  (8,2) => 170, (8,3) => 300, (8,4) => 90,
    (9,1) => 70,  (9,2) => 190, (9,3) => 350, (9,4) => 100
)

# Dynamic load at Bus 3 (stochastic, per scenario and time period)
D3 = Dict(
    (1,1) => 30,  (1,2) => 60, (1,3) => 100, (1,4) => 30,
    (2,1) => 35,  (2,2) => 70, (2,3) => 110, (2,4) => 35,
    (3,1) => 40,  (3,2) => 90, (3,3) => 120, (3,4) => 40,
    (4,1) => 30,  (4,2) => 60, (4,3) => 100, (4,4) => 30,
    (5,1) => 35,  (5,2) => 70, (5,3) => 110, (5,4) => 35,
    (6,1) => 40,  (6,2) => 90, (6,3) => 120, (6,4) => 40,
    (7,1) => 30,  (7,2) => 60, (7,3) => 100, (7,4) => 30,
    (8,1) => 35,  (8,2) => 70, (8,3) => 110, (8,4) => 35,
    (9,1) => 40,  (9,2) => 90, (9,3) => 120, (9,4) => 40
)

# Dynamic load at Bus 4 (stochastic, per scenario and time period)
D4 = Dict(
    (1,1) => 40,  (1,2) => 75, (1,3) => 120, (1,4) => 50,
    (2,1) => 50,  (2,2) => 90, (2,3) => 130, (2,4) => 60,
    (3,1) => 60,  (3,2) => 100, (3,3) => 150, (3,4) => 70,
    (4,1) => 40,  (4,2) => 75, (4,3) => 120, (4,4) => 50,
    (5,1) => 50,  (5,2) => 90, (5,3) => 130, (5,4) => 60,
    (6,1) => 60,  (6,2) => 100, (6,3) => 150, (6,4) => 70,
    (7,1) => 40,  (7,2) => 75, (7,3) => 120, (7,4) => 50,
    (8,1) => 50,  (8,2) => 90, (8,3) => 130, (8,4) => 60,
    (9,1) => 60,  (9,2) => 100, (9,3) => 150, (9,4) => 70
)

# Conventional generation parameters at Bus 1
Gmax = 600.0   # Maximum generation capacity at Bus 1
c_g = 10.0     # Generation cost per unit
c_curt = 0.0 # Penalty cost per unit of wind curtailment --> as of now doesn't make sense to include since demand, production and line capacities are not balanced yet

# Penalty cost for load shedding (typically set high)
c_ls = 100.0

# Ramping limits for conventional generation (per time period)
ramp_up = 50.0
ramp_down = 50.0

# Network parameters for DC load flow
X51 = 1.0  # Reactance on line Bus 5-1
X12 = 1.0  # Reactance on line Bus 1-2
X23 = 1.0  # Reactance on line Bus 2-3
X24 = 1.0  # Reactance on line Bus 2-4
Fmax = 500.0  # Flow limits on all lines

500.0

In [80]:
# ------------------------------
# Model Definition
# ------------------------------

model = Model(HiGHS.Optimizer)

# ------------------------------
# Constraints
# ------------------------------

# Conventional generation at Bus 1 (indexed by scenario and time)
@variable(model, g_conv[s in scenarios, t in T] >= 0, upper_bound = Gmax)

# Wind generation at Bus 5 (indexed by scenario and time)
@variable(model, g_wind[s in scenarios, t in T] >= 0)
# Wind curtailment at Bus 5 (indexed by scenario and time)
@variable(model, c_w[s in scenarios, t in T] >= 0)
# Enforce that wind generation plus curtailment equals available wind:
for s in scenarios, t in T
    @constraint(model, g_wind[s,t] + c_w[s,t] == W[(s,t)])
end

# ------------------------------
# Network Flow Variables and Voltage Angles
# ------------------------------

# Network flows on each line (indexed by scenario and time)
@variable(model, f51[s in scenarios, t in T])
@variable(model, f12[s in scenarios, t in T])
@variable(model, f23[s in scenarios, t in T])
@variable(model, f24[s in scenarios, t in T])

# Voltage angles at buses 1 through 5 (indexed by scenario and time)
@variable(model, theta[bus in 1:5, s in scenarios, t in T])
# Set reference angle at Bus 1 to 0 for all scenarios and time periods
for s in scenarios, t in T
    @constraint(model, theta[1,s,t] == 0)
end

# ------------------------------
# Load Shedding Variables
# ------------------------------
@variable(model, LS1[s in scenarios, t in T] >= 0)
@variable(model, LS2[s in scenarios, t in T] >= 0)
@variable(model, LS3[s in scenarios, t in T] >= 0)
@variable(model, LS4[s in scenarios, t in T] >= 0)

# ------------------------------
# Power Balance Constraints
# ------------------------------

# Bus 5 (Wind Bus): All wind generation flows out on the line 5-1.
for s in scenarios, t in T
    @constraint(model, g_wind[s,t] - f51[s,t] == 0)
end

# Bus 1 (Conventional Gen + Load D1): Generation plus inflow from Bus 5 must meet load D1 plus outflow.
for s in scenarios, t in T
    @constraint(model, g_conv[s,t] + f51[s,t] - f12[s,t] - D1[(s,t)] + LS1[s,t] == 0)
    @constraint(model, f12[s,t] <= g_conv[s,t] + f51[s,t] - D1[(s,t)] + LS1[s,t]) # Bus1's load is prioritized, and only any excess power can be transmitted downstream.
    @constraint(model, LS1[s,t] <= D1[(s,t)])
end

# Bus 2: The inflow from Bus 1 (f12) must supply load D2 and send out flows to Buses 3 and 4.
for s in scenarios, t in T
    @constraint(model, f12[s,t] - f23[s,t] - f24[s,t] - D2[(s,t)] + LS2[s,t] == 0)
    @constraint(model, f23[s,t] + f24[s,t] <= f12[s,t] - D2[(s,t)] + LS2[s,t])
    @constraint(model, LS2[s,t] <= D2[(s,t)])
end

# Bus 3: The inflow from Bus 2 plus load shedding LS3 must meet load D3.
for s in scenarios, t in T
    @constraint(model, f23[s,t] + LS3[s,t] - D3[(s,t)] == 0)
    @constraint(model, LS3[s,t] <= D3[(s,t)])
end


# Bus 4: The inflow from Bus 2 plus load shedding LS4 must meet load D4.
for s in scenarios, t in T
    @constraint(model, f24[s,t] + LS4[s,t] - D4[(s,t)] == 0)
    @constraint(model, LS4[s,t] <= D4[(s,t)])
end


# ------------------------------
# DC Power Flow Equations and Line Flow Limits
# ------------------------------

for s in scenarios, t in T
    @constraint(model, f51[s,t] == (theta[5,s,t] - theta[1,s,t]) / X51)
    @constraint(model, f12[s,t] == (theta[1,s,t] - theta[2,s,t]) / X12)
    @constraint(model, f23[s,t] == (theta[2,s,t] - theta[3,s,t]) / X23)
    @constraint(model, f24[s,t] == (theta[2,s,t] - theta[4,s,t]) / X24)
    
    @constraint(model, f51[s,t] <= Fmax)
    @constraint(model, -f51[s,t] <= Fmax)
    @constraint(model, f12[s,t] <= Fmax)
    @constraint(model, -f12[s,t] <= Fmax)
    @constraint(model, f23[s,t] <= Fmax)
    @constraint(model, -f23[s,t] <= Fmax)
    @constraint(model, f24[s,t] <= Fmax)
    @constraint(model, -f24[s,t] <= Fmax)
end

# ------------------------------
# Ramping Constraints for Conventional Generation at Bus 1
# ------------------------------

for s in scenarios, t in T[2:end]
    @constraint(model, g_conv[s,t] - g_conv[s,t-1] <= ramp_up)
    @constraint(model, g_conv[s,t-1] - g_conv[s,t] <= ramp_down)
end

# ------------------------------
# Objective Function
# ------------------------------

# Minimize the expected cost over all scenarios and time periods.
# The cost consists of:
#   - Conventional generation cost
#   - Penalty cost for wind curtailment
#   - Penalty cost for load shedding at buses 3 and 4
@objective(model, Min, 
    sum(prob[s] * (c_g * g_conv[s,t] + c_curt * c_w[s,t] + c_ls * (LS1[s,t] + LS2[s,t] + LS3[s,t] + LS4[s,t]))
        for s in scenarios, t in T)
)


1.1111111111111112 g_conv[1,1] + 11.11111111111111 LS1[1,1] + 11.11111111111111 LS2[1,1] + 11.11111111111111 LS3[1,1] + 11.11111111111111 LS4[1,1] + 1.1111111111111112 g_conv[1,2] + 11.11111111111111 LS1[1,2] + 11.11111111111111 LS2[1,2] + 11.11111111111111 LS3[1,2] + 11.11111111111111 LS4[1,2] + 1.1111111111111112 g_conv[1,3] + 11.11111111111111 LS1[1,3] + 11.11111111111111 LS2[1,3] + 11.11111111111111 LS3[1,3] + 11.11111111111111 LS4[1,3] + 1.1111111111111112 g_conv[1,4] + 11.11111111111111 LS1[1,4] + 11.11111111111111 LS2[1,4] + 11.11111111111111 LS3[1,4] + 11.11111111111111 LS4[1,4] + 1.1111111111111112 g_conv[2,1] + 11.11111111111111 LS1[2,1] + 11.11111111111111 LS2[2,1] + 11.11111111111111 LS3[2,1] + 11.11111111111111 LS4[2,1] + 1.1111111111111112 g_conv[2,2] + 11.11111111111111 LS1[2,2] + 11.11111111111111 LS2[2,2] + 11.11111111111111 LS3[2,2] + 11.11111111111111 LS4[2,2] + [[...120 terms omitted...]] + 1.1111111111111112 g_conv[8,3] + 11.11111111111111 LS1[8,3] + 11.11111111111

In [81]:
# ------------------------------
# Solving / Model Analyisis
# ------------------------------
optimize!(model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+01]
  Bound  [6e+02, 6e+02]
  RHS    [3e+01, 7e+02]
Presolving model
198 rows, 252 cols, 684 nonzeros  0s
138 rows, 222 cols, 564 nonzeros  0s
51 rows, 90 cols, 132 nonzeros  0s
50 rows, 82 cols, 123 nonzeros  0s
Presolve : Reductions: rows 50(-904); columns 82(-494); elements 123(-1749)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
         47     2.3885555556e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 47
Objective value     :  2.3885555556e+04
Relative P-D gap    :  7.6154368665e-16
HiGHS run time      :          0.00


In [84]:
println("Overall Optimal Objective Value: ", objective_value(model))

# Print results per scenario and time period
for s in scenarios, t in T
    println("Scenario $s, Time Period $t:")
    println("  Conventional Generation (Bus 1): ", value(g_conv[s,t]))
    println("  Wind Generation Used (Bus 5): ", value(g_wind[s,t]))
    println("  Wind Curtailment (Bus 5): ", value(c_w[s,t]))
    println("  Load at Bus 1: ", D1[(s,t)], "  Load Shedding (Bus 1): ", value(LS1[s,t]))
    println("  Load at Bus 2: ", D2[(s,t)], "  Load Shedding (Bus 2): ", value(LS2[s,t]))
    println("  Load at Bus 3: ", D3[(s,t)], "  Load Shedding (Bus 3): ", value(LS3[s,t]))
    println("  Load at Bus 4: ", D4[(s,t)], "  Load Shedding (Bus 4): ", value(LS4[s,t]))
end

Overall Optimal Objective Value: 23885.555555555566
Scenario 1, Time Period 1:
  Conventional Generation (Bus 1): 186.5
  Wind Generation Used (Bus 5): 33.5
  Wind Curtailment (Bus 5): 286.6
  Load at Bus 1: 100  Load Shedding (Bus 1): 0.0
  Load at Bus 2: 50  Load Shedding (Bus 2): 0.0
  Load at Bus 3: 30  Load Shedding (Bus 3): 0.0
  Load at Bus 4: 40  Load Shedding (Bus 4): 0.0
Scenario 1, Time Period 2:
  Conventional Generation (Bus 1): 236.5
  Wind Generation Used (Bus 5): 138.5
  Wind Curtailment (Bus 5): 161.10000000000002
  Load at Bus 1: 90  Load Shedding (Bus 1): 0.0
  Load at Bus 2: 150  Load Shedding (Bus 2): 0.0
  Load at Bus 3: 60  Load Shedding (Bus 3): 0.0
  Load at Bus 4: 75  Load Shedding (Bus 4): 0.0
Scenario 1, Time Period 3:
  Conventional Generation (Bus 1): 286.5
  Wind Generation Used (Bus 5): 333.5
  Wind Curtailment (Bus 5): 0.0
  Load at Bus 1: 150  Load Shedding (Bus 1): 0.0
  Load at Bus 2: 250  Load Shedding (Bus 2): 0.0
  Load at Bus 3: 100  Load Sheddin